In [ ]:
%matplotlib inline

In [92]:
import math
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import seaborn as sns
from IPython.display import display, HTML
from brainflow.data_filter import DataFilter, AggOperations, WaveletTypes, NoiseEstimationLevelTypes, WaveletExtensionTypes, ThresholdTypes, WaveletDenoisingTypes
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, NoiseTypes, DetrendOperations, WindowOperations
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

In [93]:
board_id = BoardIds.UNICORN_BOARD
board = BoardShim(board_id, BrainFlowInputParams())

In [94]:
channels = board.get_eeg_channels(board_id) #EEG Channels
timestamp_channel = board.get_timestamp_channel(board_id) # Timestamp channel
marker_channel = board.get_marker_channel(board_id) # Marker channel for synchronization
sampling_rate = BoardShim.get_sampling_rate(board_id) # Hz
print("Channels: ", channels)
print("Timestamp: ", timestamp_channel)
print("Marker: ", marker_channel)
print("Sampling Rate: ", sampling_rate)

Channels:  [0, 1, 2, 3, 4, 5, 6, 7]
Timestamp:  17
Marker:  18
Sampling Rate:  250


In [95]:
df = pd.read_csv('data/eeg_data1.csv', header=None)

In [96]:
x_ = df[timestamp_channel]
Oz = df[channels[6]]
Po7 = df[channels[5]]
Po8 = df[channels[7]]

eeg_data = Po7.to_numpy() - Oz.to_numpy()

In [97]:
filtered_eeg = eeg_data
DataFilter.perform_bandpass(filtered_eeg, sampling_rate, 2.0, 30.0, 2, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)
DataFilter.perform_bandstop(filtered_eeg, sampling_rate, 58.0, 62.0, 5, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)

In [98]:
first = df.loc[df[marker_channel] == 1].index[0]
target_eeg_18hz = filtered_eeg[first:first+sampling_rate*20]
target_time_18hz = x_[first:first+sampling_rate*20]
sns.lineplot(x=target_time_18hz,y=target_eeg_18hz)
plt.savefig('figures/eeg18hz.jpg')
plt.close()

base_first = df.loc[df[marker_channel] == 2].index[0]
base_eeg_18hz = filtered_eeg[base_first:base_first+sampling_rate*20]
base_time_18hz = x_[base_first:base_first+sampling_rate*20]
sns.lineplot(x=base_time_18hz,y=base_eeg_18hz)
plt.savefig('figures/base18hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/eeg18hz.jpg'></td><td><img src='figures/base18hz.jpg'></td></tr></table>"))

,


In [99]:
nfft = DataFilter.get_nearest_power_of_two(sampling_rate)

target_psd_18hz = DataFilter.get_psd_welch(target_eeg_18hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)
base_psd_18hz = DataFilter.get_psd_welch(base_eeg_18hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)

sns.lineplot(x=target_psd_18hz[1],y=target_psd_18hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_eeg18hz.jpg')
plt.close()
sns.lineplot(x=base_psd_18hz[1],y=base_psd_18hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_base18hz.jpg')
plt.close()

diff_t = (base_psd_18hz[1]+target_psd_18hz[1])/2
diff_y = target_psd_18hz[0]-base_psd_18hz[0]
print(diff_t[np.argmax(diff_y)])
sns.lineplot(x=diff_t,y=diff_y)
plt.xlim(0, 40)
plt.savefig('figures/psd_diff18hz.jpg')
plt.close()


display(HTML(f"<table><tr><td><img src='figures/psd_eeg18hz.jpg'></td><td><img src='figures/psd_base18hz.jpg'></td></tr></table>"))

17.578125


,


In [100]:
second = df.loc[df[marker_channel] == 3].index[0]
target_eeg_16hz = filtered_eeg[second:second+sampling_rate*20]
target_time_16hz = x_[second:second+sampling_rate*20]
sns.lineplot(x=target_time_16hz, y=target_eeg_16hz)
plt.savefig('figures/eeg16hz.jpg')
plt.close()

base_second = df.loc[df[marker_channel] == 4].index[0]
base_eeg_16hz = filtered_eeg[base_second:base_second+sampling_rate*20]
base_time_16hz = x_[base_second:base_second+sampling_rate*20]
sns.lineplot(x=base_time_16hz,y=base_eeg_16hz)
plt.savefig('figures/base16hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/eeg16hz.jpg'></td><td><img src='figures/base16hz.jpg'></td></tr></table>"))

,


In [101]:
target_psd_16hz = DataFilter.get_psd_welch(target_eeg_16hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)
base_psd_16hz = DataFilter.get_psd_welch(base_eeg_16hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)

sns.lineplot(x=target_psd_16hz[1],y=target_psd_16hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_eeg16hz.jpg')
plt.close()
sns.lineplot(x=base_psd_16hz[1],y=base_psd_16hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_base16hz.jpg')
plt.close()

diff_t = (base_psd_16hz[1]+target_psd_16hz[1])/2
diff_y = target_psd_16hz[0]-base_psd_16hz[0]
print(diff_t[np.argmax(diff_y)])
sns.lineplot(x=diff_t,y=diff_y)
plt.xlim(0, 40)
plt.savefig('figures/psd_diff16hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/psd_eeg16hz.jpg'></td><td><img src='figures/psd_base16hz.jpg'></td></tr></table>"))

15.625


,


In [102]:
third = df.loc[df[marker_channel] == 5].index[0]
target_eeg_9hz = filtered_eeg[third:third+sampling_rate*20]
target_time_9hz = x_[third:third+sampling_rate*20]
sns.lineplot(x=target_time_9hz,y=target_eeg_9hz)
plt.savefig('figures/eeg9hz.jpg')
plt.close()

base_third = df.loc[df[marker_channel] == 6].index[0]
base_eeg_9hz = filtered_eeg[base_third:base_third+sampling_rate*20]
base_time_9hz = x_[base_third:base_third+sampling_rate*20]
sns.lineplot(x=base_time_9hz,y=base_eeg_9hz)
plt.savefig('figures/base9hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/eeg9hz.jpg'></td><td><img src='figures/base9hz.jpg'></td></tr></table>"))

,


In [103]:
target_psd_9hz = DataFilter.get_psd_welch(target_eeg_9hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)
base_psd_9hz = DataFilter.get_psd_welch(base_eeg_9hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)

sns.lineplot(x=target_psd_9hz[1],y=target_psd_9hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_eeg9hz.jpg')
plt.close()
sns.lineplot(x=base_psd_9hz[1],y=base_psd_9hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_base9hz.jpg')
plt.close()

diff_t = (base_psd_9hz[1]+target_psd_9hz[1])/2
diff_y = target_psd_9hz[0]-base_psd_9hz[0]
print(diff_t[np.argmax(diff_y)])
sns.lineplot(x=diff_t,y=diff_y)
plt.xlim(0, 40)
plt.savefig('figures/psd_diff9hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/psd_eeg9hz.jpg'></td><td><img src='figures/psd_base9hz.jpg'></td></tr></table>"))

8.7890625


,


In [104]:
fourth = df.loc[df[marker_channel] == 7].index[0]
target_eeg_24hz = filtered_eeg[fourth:fourth+sampling_rate*20]
target_time_24hz = x_[fourth:fourth+sampling_rate*20]
sns.lineplot(x=target_time_24hz,y=target_eeg_24hz)
plt.savefig('figures/eeg24hz.jpg')
plt.close()

base_fourth = base_third
base_eeg_24hz = filtered_eeg[base_fourth:base_fourth+sampling_rate*20]
base_time_24hz = x_[base_fourth:base_fourth+sampling_rate*20]
sns.lineplot(x=base_time_24hz,y=base_eeg_24hz)
plt.savefig('figures/base24hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/eeg24hz.jpg'></td><td><img src='figures/base24hz.jpg'></td></tr></table>"))


,


In [105]:
target_psd_24hz = DataFilter.get_psd_welch(target_eeg_24hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)
base_psd_24hz = DataFilter.get_psd_welch(base_eeg_24hz, nfft, nfft // 2, sampling_rate, WindowOperations.BLACKMAN_HARRIS.value)

sns.lineplot(x=target_psd_24hz[1],y=target_psd_24hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_eeg24hz.jpg')
plt.close()
sns.lineplot(x=base_psd_24hz[1],y=base_psd_24hz[0])
plt.xlim(0, 40)
plt.savefig('figures/psd_base24hz.jpg')
plt.close()

diff_t = (base_psd_24hz[1]+target_psd_24hz[1])/2
diff_y = target_psd_24hz[0]-base_psd_24hz[0]
print(diff_t[np.argmax(diff_y)])
sns.lineplot(x=diff_t,y=diff_y)
plt.xlim(0, 40)
plt.savefig('figures/psd_diff24hz.jpg')
plt.close()

display(HTML(f"<table><tr><td><img src='figures/psd_eeg24hz.jpg'></td><td><img src='figures/psd_base24hz.jpg'></td></tr></table>"))

23.4375


,
